In [1]:
%matplotlib inline
%config InLineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import os
import glob
import matplotlib.pyplot as plt
import random
import numpy as np

In [9]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
DIR = 'C:/Users/gulte/Desktop/Bitirme Ödevi/images'

In [3]:
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
    
])

In [4]:
test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
])

In [5]:
train_data = datasets.ImageFolder(DIR + '/training', transform=train_transforms)
test_data = datasets.ImageFolder(DIR + '/test', transform=test_transforms)

In [13]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size = 64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size = 64)

In [6]:
model = models.densenet121(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\gulte/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100.0%


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024,512),
                                nn.ReLU(),
                                nn.Dropout(0.2),
                                nn.Linear(512,256),
                                nn.ReLU(),
                                nn.Dropout(0.1),
                                nn.Linear(256,4),
                                nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [25]:
traininglosses = []
testinglosses = []
testaccuracy = []
totalsteps = []
epochs = 1
steps = 0
running_loss = 0
print_every = 5

In [26]:
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            traininglosses.append(running_loss/print_every)
            testinglosses.append(test_loss/len(testloader))
            testaccuracy.append(accuracy/len(testloader))
            totalsteps.append(steps)
            
            print(f"Device {device}.."
                 f"Epochs {epochs+1}/{epochs}"
                 f"Step {steps}"
                 f"Train loss: {running_loss/print_every:.3f}"
                 f"Test loss: {test_loss/len(testloader):.3f}"
                 f"TEST ACC: {accuracy/len(testloader)}")
            running_loss = 0
            model.train()

Device cuda..Epochs 2/1Step 5Train loss: 0.148Test loss: 0.122TEST ACC: 0.9767361106695952
Device cuda..Epochs 2/1Step 10Train loss: 0.617Test loss: 0.137TEST ACC: 0.9488425917095609
Device cuda..Epochs 2/1Step 15Train loss: 0.333Test loss: 0.257TEST ACC: 0.9221064823645132
Device cuda..Epochs 2/1Step 20Train loss: 0.199Test loss: 0.123TEST ACC: 0.9516203712534022
Device cuda..Epochs 2/1Step 25Train loss: 0.169Test loss: 0.108TEST ACC: 0.9620370379200688
Device cuda..Epochs 2/1Step 30Train loss: 0.155Test loss: 0.082TEST ACC: 0.9748842583762275
Device cuda..Epochs 2/1Step 35Train loss: 0.195Test loss: 0.087TEST ACC: 0.9708333324503016
Device cuda..Epochs 2/1Step 40Train loss: 0.082Test loss: 0.074TEST ACC: 0.9737268509688201
Device cuda..Epochs 2/1Step 45Train loss: 0.050Test loss: 0.058TEST ACC: 0.9771990731910423
Device cuda..Epochs 2/1Step 50Train loss: 0.098Test loss: 0.044TEST ACC: 0.9800925917095609
Device cuda..Epochs 2/1Step 55Train loss: 0.051Test loss: 0.048TEST ACC: 0.981365

In [ ]:
from matplotlib import pyplot as plt
plt.plot(totalsteps, traininglosses, label = 'Train Loss')
plt.plot(totalsteps, testinglosses, label = 'Test Loss')
plt.plot(totalsteps, testaccuracy, label = 'Test Acc')
plt.legend()
plt.grip()
plt.show()